<a href="https://colab.research.google.com/github/sarmientoj24/EE298/blob/master/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import os

In [0]:
# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
  z_mean, z_log_var = args
  batch = K.shape(z_mean)[0]
  dim = K.int_shape(z_mean)[1]
  # by default, random_normal has mean=0 and std=1.0
  epsilon = K.random_normal(shape=(batch, dim))
  return z_mean + K.exp(0.5 * z_log_var) * epsilon


# DISCRETE EPSILON for Code Generation
# To Use:
# Encoder's last layer should output z_mean, z_log_var, and possible z.
# But on encoding dataset in Code Generation, we disregard z and use this function
# to compute for a discrete z using z_mean and z_log_var
def discrete_z_sampling(args)
  z_mean, z_log_var, epsilon_discrete = args

  return z_mean + K.exp(0.5 * z_log_var) * epsilon_discrete

def randomly_sample_epsilon_discrete(dimension):
  epsilon_discrete = K.random_normal(shape=(1, dimension))
  print("Generating discrete epsilon...")
  print(epsilon_discrete)
  return epsilon_discrete


In [0]:
# Plotter functions here

In [0]:
# Implement VAE here

class VAE():
  pass

  def save_encoder_to_h5(self):
    pass
  
  def save_decoder_to_h5(self):
    pass
  
  def save_vae_to_h5(self):
    pass